In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from requests_html import HTMLSession
import time
import asyncio
import aiohttp
import aiofiles
import os

In [2]:
nps = 'https://calhoun.nps.edu/handle/10945/16/discover?search-result=true&query=&current-scope=10945%2F16&filtertype_0=type&filtertype_1=dateIssued&filter_relational_operator_1=equals&filter_relational_operator_0=equals&filter_1=%5B2020+TO+2022%5D&filter_0=Thesis&rpp=100&sort_by=score&order=desc'

In [3]:
jfq = 'https://ndupress.ndu.edu/JFQ/'
dod = 'https://irp.fas.org/doddir/dod/index.html'
rand = 'https://www.rand.org/content/dam/rand/pubs/occasional_papers/2011/RAND_OP326.pdf'

In [4]:
#!/usr/bin/env python3

from requests_html import HTMLSession
import time
import asyncio
import aiohttp
import aiofiles
import os

MAX_PARALLELISM = 10
REPORTS_FOLDER = "/Users/americanthinker1/NationalSecurityBERT/Data/pretraining/CJCS"

def download_files(urls):
    #os.mkdir(REPORTS_FOLDER)
    sema = asyncio.BoundedSemaphore(MAX_PARALLELISM)

    async def fetch_file(url):
        fname = url.split("/")[-1]
        async with sema, aiohttp.ClientSession() as session:
            async with session.get(url) as resp:
                assert resp.status == 200
                data = await resp.read()

        async with aiofiles.open(
            os.path.join(REPORTS_FOLDER, fname), "wb"
        ) as outfile:
            await outfile.write(data)

        print("file downloaded : {}".format(fname))

    loop = asyncio.get_event_loop()
    tasks = [loop.create_task(fetch_file(url)) for url in urls]
    loop.run_until_complete(asyncio.wait(tasks))
    loop.close()
    
cjcs = 'https://www.jcs.mil/Library/CJCS-Instructions/'

session = HTMLSession()
r = session.get(cjcs)

links_raw = r.html.absolute_links
pdf_links = [x for x in links_raw if x.endswith('pdf')]

#pdf_links_noexcerpt = [x for x in pdf_links if 'excerpt' not in x]
#print("\n".join(pdf_links_noexcerpt))
print("\n".join(pdf_links))
download_files(pdf_links)

RuntimeError: This event loop is already running